In [1]:
import os
import glob
import re
import numpy as np
import numpy.fft as fft
import mne
import matplotlib.pyplot as plt
from scipy import stats, signal
from numpy import save, load
import pandas as pd
from sklearn.svm import SVC

### read dataset X and Y

In [2]:
subject_id=1
base_path = "features/"
edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb01/*.npy".format(subject_id))))
files=len(edf_file_names)
X=load('features/data_chb01/features_{}_00.npy'.format(subject_id))
y=load('features/data_chb01/targets_{}_00.npy'.format(subject_id))
print(X.shape)
for fileno in range(1, files//2):
    X=np.concatenate((X, load('features/data_chb01/features_{}_{:02d}.npy'.format(subject_id, fileno))))
    y=np.concatenate((y, load('features/data_chb01/targets_{}_{:02d}.npy'.format(subject_id, fileno))))


(408, 208)


In [3]:
X.shape, y.shape

((14584, 208), (14584,))

In [4]:
df=pd.DataFrame(data=X)

In [5]:
df['target']=y

In [6]:
df.shape

(14584, 209)

In [20]:
df.dropna()
df.shape

(13163, 209)

In [16]:
df[(df['target']==3) | (df['target']==1)]

,0,1,2,3,4,5,6,7,8,9,...,199,200,201,202,203,204,205,206,207,target
767,0.504019,0.161749,0.036241,0.087788,0.117071,0.010672,0.075750,0.006710,35.378636,0.340825,...,0.282103,0.070293,0.147826,0.065048,0.009173,0.059392,0.005359,22.022274,1.0,3
768,0.397320,0.131061,0.041063,0.131679,0.127953,0.018591,0.123891,0.028443,41.653349,0.137482,...,0.261818,0.062877,0.153241,0.082529,0.013847,0.073221,0.010040,24.051126,1.0,3
769,0.032970,0.067123,0.011638,0.045629,0.222607,0.082048,0.388199,0.149785,359.949214,0.022584,...,0.168848,0.029229,0.121630,0.179059,0.042258,0.250080,0.064458,80.585204,1.0,3
770,0.013406,0.067360,0.010145,0.044625,0.228867,0.084977,0.397326,0.153293,342.790860,0.017106,...,0.153139,0.030927,0.122526,0.180385,0.046731,0.282134,0.073164,88.751985,1.0,3
771,0.131704,0.184924,0.037553,0.238695,0.201855,0.023583,0.158190,0.023495,23.179371,0.198627,...,0.184617,0.045994,0.168155,0.154820,0.037461,0.221824,0.067128,31.520675,1.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9151,0.024826,0.082295,0.049787,0.065915,0.295219,0.086339,0.333806,0.061814,56.292130,0.102190,...,0.048700,0.042667,0.101418,0.115535,0.100833,0.517067,0.014999,97.952720,1.0,3
9152,0.014045,0.083170,0.027623,0.031817,0.319929,0.085996,0.365671,0.071750,75.450678,0.065839,...,0.056387,0.047088,0.089704,0.125709,0.098840,0.507692,0.015631,98.250833,1.0,3
9153,0.022369,0.120699,0.032243,0.047050,0.325650,0.063891,0.319887,0.068213,45.941663,0.062855,...,0.053789,0.052068,0.111783,0.115940,0.098990,0.508954,0.013975,97.544856,1.0,3
9154,0.034451,0.159178,0.060282,0.076829,0.325747,0.056455,0.239840,0.047217,27.680383,0.085372,...,0.045370,0.047184,0.114786,0.107574,0.100771,0.512889,0.013147,94.074203,1.0,3


In [19]:
df=df[(df['target']==2) | (df['target']==0)]
#discard discarded values

In [18]:
df.shape

(14584, 209)

### train test split

In [35]:
df_interictal=df[df['target']==0]
df_preictal=df[df['target']==2]
df_interictal.shape, df_preictal.shape

((12188, 209), (975, 209))

In [39]:
df_interictal['target']

0        0
1        0
2        0
3        0
4        0
        ..
14579    0
14580    0
14581    0
14582    0
14583    0
Name: target, Length: 12188, dtype: int64

In [40]:
X_interictal=np.array(df_interictal[df.columns[:-1]])
y_interictal=np.array(df_interictal['target'])
X_preictal  =np.array(df_preictal[df.columns[:-1]])
y_preictal  =np.array(df_preictal['target'])

In [42]:
print(X_interictal.shape)
print(y_interictal.shape)
print(X_preictal.shape)
print(y_preictal.shape)

(12188, 208)
(12188,)
(975, 208)
(975,)


In [ ]:
df_preictal[:-1]

In [24]:
from sklearn.model_selection import train_test_split


In [43]:
X_interictal_train, X_interictal_test, y_interictal_train, y_interictal_test =train_test_split(X_interictal,y_interictal,test_size=0.1, random_state=42)
X_preictal_train, X_preictal_test, y_preictal_train, y_preictal_test=train_test_split(X_preictal, y_preictal,test_size=0.1, random_state=42)

In [46]:
X_train = np.concatenate((X_interictal_train, X_preictal_train))
X_test = np.concatenate((X_interictal_test, X_preictal_test))
y_train = np.concatenate((y_interictal_train, y_preictal_train))
y_test = np.concatenate((y_interictal_test, y_preictal_test))

In [47]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(11846, 208)
(1317, 208)
(11846,)
(1317,)


### training the model

In [48]:
clf = SVC()
clf.fit(X_train, y_train)
trainAcc = clf.score(X_train, y_train)
testAcc = clf.score(X_test, y_test)
print("**SVM Results:**")
print("Training Accuracy: %d"%(trainAcc*100)+"%")
print("Testing Accuracy: %d"%(testAcc *100)+"%")

**SVM Results:**
Training Accuracy: 92%
Testing Accuracy: 92%
